# <center> TD2 Modèlisation Mathématique </center>


Nous allons développer cette semaine l'algorithme des *k* plus proches voisins.

Nous allons d'abord le développer à la main. 

> *Dans le monde professionnel, il ne faudra surtout pas utiliser un programme de classification développé à la main car il ne sera pas optimisé. Nous le réalisons ici dans un but pédagogique afin de bien comprendre comment il fonctionne et pour le maîtriser.*

Nous verrons en fin de ce TD comment utiliser la bibliothèque *scikit-learn* pour mettre en place ce classifieur de manière très aisée.

L'algorithme des *k plus proches voisins* est un algorithme de classification.  
Il n'y a pas à proprement parlé dans cet algorithme de phase d'apprentissage comme nous en verrons dans d'autres algorithmes. 
L'ensemble d'apprentissage correspond aux données qui sont classées. L'ensemble de test est constitué de données à classer pour tester l'algorithme.

La phase de prédiction consiste pour chaque élément de l'ensemble de test à prédire son étiquette en déterminant la classe majoritaire de ses *k* plus proches voisins qui sera alors la sienne.

Nous allons travailler avec un jeu de données "jouet" conçu pour tester les algorithmes sur des exemples simples (*make_circles*).




In [2]:
#chargement des bibliothèques python

import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
# pour générer le jeu de données
from sklearn.datasets import make_circles
#pour calculer l'exactitude (accuracy) de la prédiction
from sklearn.metrics import accuracy_score



### 1) Mise en place des données

Nous allons ici générer le jeu de données. Nous aurons:
 - les données et les étiquettes d'apprentissage: *x_train* et *y_train* (de taille 100)
 - les données et les étiquettes de test: *x_test* et *y_test* (de taille 50)

 Ce sont ces données de test que nous classifierons (*x_test*) en nous appuyant sur la classification des données d'apprentissage.
 
 Nous évaluerons la classification obtenue des élèments de *x_test* par rapport à la classification réelle de ces élèments (*y_test*)


In [ ]:
#génération des ensembles de données et des étiquettes, regarder la documentation de make_circles

x_train , y_train = make_circles(100, noise=0, random_state=1)
x_test , y_test = make_circles(50, noise=0.1, random_state=1)
print(x_train)

Nous allons maintenant visualiser ces données (apprentissage et test). Utiliser plt.scatter(...) pour visualiser tous les points.

In [ ]:
#TODO

plt.scatter(x_train[y_train==0][:, 0], x_train[y_train==0][:, 1], c='green')
plt.scatter(x_train[y_train==1][:, 0], x_train[y_train==1][:, 1], c='orange')

plt.scatter(x_test[y_test==0][:, 0], x_test[y_test==0][:, 1], c='pink')
plt.scatter(x_test[y_test==1][:, 0], x_test[y_test==1][:, 1], c='black')


### 2) Développement de la classe *KnnClassifieur*

Nous allons écrire une classe *python* nommée *KnnClassifieur* qui va nous permettre de réaliser le classifieur.




In [8]:
from scipy.stats import mode

class KnnClassifieur:

#constructeur
#parametre:
#k: le nombre de voisins 
#x_train: contient les éléments dont on connait la classe, 1 ligne = 1 élèment
#y_train: contient les étiquettes des classes des éléments de x_train qui sont des entiers


  def __init__(self, k: int,x_train: np.ndarray, y_train: np.ndarray):
    self.k = k
    self.x_train = x_train
    self.y_train = y_train


  @staticmethod
  #calcule la distance euclidienne entre 2 vecteurs x et y
  #Parametres:
  #x: le premier vecteur
  #y: le second vecteur
  #Retour:
  #la distance euclidienne entre x et y

  def distance_euclidienne(x: np.ndarray, y: np.ndarray):
    return np.sqrt(np.sum(np.square(x-y)))

  
#prédit l'étiquette de la classe d'un élèment du test
#Parametre:
#une ligne de l'ensemble de test qui correspond à l'élément à classer
#Retour
#l'étiquette de la classe prédite qui est un entier

  def predict_etiquette(self, x_test_ligne: np.ndarray) -> int:
    distances=[]
    #on itère sur les lignes du x_train
    for j in range(len(self.x_train)):
      distance=KnnClassifieur.distance_euclidienne(x_test_ligne,self.x_train[j,:])
      distances.append(distance)
    distances=np.array(distances)
    #distances_k_plus_proche est un tableau d'une ligne qui contient les k plus proches voisins (ce sont les indices qui sont stockés)
    #sa taille est de k
    distances_k_plus_proche=np.argsort(distances)[:self.k]
    #on recupere les étiquettes de ces k plus proches voisins
    etiquettes=self.y_train[distances_k_plus_proche]
    #détermination de l'étiquette majoritaire dans etiquettes
    etiquette=mode(etiquettes, keepdims=True)
    #on ne récupère que l'étiquette majoritaire
    etiquette=etiquette.mode[0]
    return etiquette

    
#prédit toutes les étiquettes des classes des éléments de x_test
#Parametres:
#x_test: les éléments à classer
#Retour:
#un tableau contenant les étiquettes des classes prédites des élèments de x_test

  def predict(self, x_test: np.ndarray) -> np.ndarray:
    #return np.array(list(map(lambda var : self.predict_etiquette(var), x_test)))
    etiquettes_prediction=[]
    for x in x_test:
     etiquette=self.predict_etiquette(x)
     etiquettes_prediction.append(etiquette)
    return np.array(etiquettes_prediction)





### 3) Utilisation du classifieur

a) Vous réaliserez les prédictions des étiquettes des classes sur les données *x_test* 

b) Vous représenterez graphiquement les données d'apprentissage et de test avec les étiquettes des classes prédites

c) Vous calculerez le score d'exactitude (accuracy) de votre prédiction. Les données de test n'étant pas très bruitées vous devez obtenir 1.  
Modifier le paramètre *noise* de la fonction qui a permis de générer les données de test, pour obtenir des données plus bruitées (ce qui va dégrader les résultats).
Afficher aussi les valeurs de la précision et du rappel. 

d) Vous générerez la matrice de confusion et recalculerez les valeurs de l'exactitude, de la précision et du rappel (avec les formules vues en cours). Vous devez retrouver les résultats précédents.

In [9]:
knn = KnnClassifieur(3, x_train,y_train)
prediction = knn.predict(x_test)



In [ ]:
plt.scatter(x_train[y_train==0][:, 0], x_train[y_train==0][:, 1], c='violet')
plt.scatter(x_train[y_train==1][:, 0], x_train[y_train==1][:, 1], c='yellow')

plt.scatter(x_test[prediction==0][:, 0], x_test[prediction==0][:, 1], c='indigo')
plt.scatter(x_test[prediction==1][:, 0], x_test[prediction==1][:, 1], c='goldenrod')


In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

print(accuracy_score(y_test, prediction))
print(precision_score(y_test,prediction))
print(recall_score(y_test,prediction))

print(confusion_matrix(y_test,prediction))
tn,fp,fn,tp=confusion_matrix(y_test, prediction).ravel()
accuracy=(tp+tn)/(tp+fn+tn+fp)
print(accuracy)
precision=tp/(tp+fp)
print(precision)
rappel=tp/(tp+fn)
print(rappel)